In [1]:
import json
import os
import xml.etree.ElementTree as ET

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

Using <_io.TextIOWrapper name='./maap.cfg' mode='r' encoding='UTF-8'> as maap config file


Set up workflow config

In [7]:
input_config =  [
      {
        "step_key": "l1_preprocess",
        "algorithm_id": "sister-preprocess",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "params": {
          "raw_dataset": "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL1BRAD.001/EMIT_L1B_RAD_001_20220810T174213_2222212_001/EMIT_L1B_RAD_001_20220810T174213_2222212_001.nc",
          "crid": 999,
          "identifier": "SISTER_EMIT_L1B_RDN_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_reflectance",
        "algorithm_id": "sister-isofit",
        "queue": "sister-job_worker-32gb",
        "version": "sister-dev",
        "input_filter": {
          "radiance_dataset": "*RDN*999",
          "location_dataset": "*RDN*LOC",
          "observation_dataset": "*RDN*OBS"
        },
        "params": {
          "radiance_dataset": None,
          "location_dataset": None,
          "observation_dataset": None,
          "n_cores": 32,
          "segmentation_size": 50,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2A_RFL_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_resample",
        "algorithm_id": "sister-resample",
        "version": "2.0.0",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "reflectance_dataset": "*_RFL_*999",
          "uncertainty_dataset": "*_RFL*UNC"
        },
        "params": {
          "reflectance_dataset": None,
          "uncertainty_dataset": None,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2A_RSRFL_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_reflectance_correction",
        "algorithm_id": "sister-reflect_correct",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "observation_dataset": "*RDN*OBS",
          "reflectance_dataset": "*RSRFL*999"
        },
        "params": {
          "observation_dataset": None,
          "reflectance_dataset": None,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2A_CORFL_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_factional_cover",
        "algorithm_id": "sister-fractional-cover",
        "version": "1.0.0",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "reflectance_dataset": "*CORFL*"
        },
        "params": {
          "reflectance_dataset": None,
          "n_cores": 10,
          "refl_scale": 1,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_FRCOVER_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_vegbiochem",
        "algorithm_id": "sister-trait_estimate",
        "version": "1.0.0",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "reflectance_dataset": "*CORFL*",
          "frcov_dataset": "*L2B_FRCOV*"
        },
        "params": {
          "reflectance_dataset": None,
          "frcov_dataset": None,
          "veg_cover": 0.5,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_VEGBIOCHEM_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_grainsize",
        "algorithm_id": "sister-grainsize",
        "version": "1.0.0",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "reflectance_dataset": "*CORFL*",
          "frcov_dataset": "*L2B_FRCOV*"
        },
        "params": {
          "reflectance_dataset": None,
          "frcov_dataset": None,
          "snow_cover": 0.9,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_GRAINSIZE_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_aquatic_pigments",
        "algorithm_id": "sister-aquatic-pigments-pge",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "corrected_reflectance_dataset": "*CORFL*",
          "fractional_cover_dataset": "*L2B_FRCOV*"
        },
        "params": {
          "corrected_reflectance_dataset": None,
          "fractional_cover_dataset": None,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_AQUAPIG_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_benthic_inversion",
        "algorithm_id": "sister-benthic-inversion-pge",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "corrected_reflectance_dataset": "*CORFL*",
          "fractional_cover_dataset": "*L2B_FRCOV*"
        },
        "params": {
          "corrected_reflectance_dataset": None,
          "fractional_cover_dataset": None,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_BENTHRFL_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "l2_benthic_cover",
        "algorithm_id": "sister-benthic-cover-pge",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "input_filter": {
          "benthic_reflectance_dataset": "*BENTHRFL*",
          "fractional_cover_dataset": "*L2B_FRCOV*",
          "depth_dataset": "*DEPTH"
        },
        "params": {
          "benthic_reflectance_dataset": None,
          "fractional_cover_dataset": None,
          "depth_dataset": None,
          "crid": 999,
          "identifier": "SISTER_EMIT_L2B_BENTHCOVER_20220810T174213_999",
          "username": "anonymous"
        }
      },
      {
        "step_key": "sister_router",
        "snow_cover": 0.5,
        "veg_cover": 0.5,
        "min_pixels": 100,
        "soil_cover": 0.5,
        "water_cover": 0.5
      }
    ]


Submit the workflow job

In [11]:
job = maap.submitJob(
    algo_id="run_sister_workflow",
    version="dev",
    queue="sister-job_worker-8gb",
    identifier="sister-cwl-workflow-test",
    username="anonymous",
    workflow_config=json.dumps(input_config),
    precondition_file="sister_router.py"
),
job


({'job_id': '95b448ff-0629-4dd3-8bb6-e57f30b0b66b', 'status': 'success', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []},)